## Initial setup

Our application will use three components:
- A LLM model, to chat with
- An Embedding model
- A Vector Store

Let's initialize them at the very beginning, to have the entire architecture in place.

In [8]:
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_ollama.chat_models import ChatOllama

OLLAMA_BASEURL='http://192.168.1.99:11434/'
OLLAMA_MODEL='llama3.2:latest'

llm = ChatOllama(
        model = OLLAMA_MODEL,
        temperature = 0,
        base_url = OLLAMA_BASEURL
    )

embeddings = OllamaEmbeddings(
    model=OLLAMA_MODEL,
    base_url = OLLAMA_BASEURL
)

vector_store = InMemoryVectorStore(embeddings)

## Knowledge base population

Let's add the documents which constitute our knowledge base into the Vectore Store.

In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

def load_to_knowledge(file_path):
    print(f"\nProcessing {file_path}")
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    print(f'\t{len(docs)} pages found')

    splits = text_splitter.split_documents(docs)
    print(f'\t{len(splits)} splits generated')

    print('Saving splits to Vectore Store...')
    vector_store.add_documents(documents=splits)
    print('\tSaved.\n')

In [10]:
knowledge_base_items = [
    'docs/SIRA_CV_EN.pdf'
]

for item in knowledge_base_items:
    load_to_knowledge(item)

print('Knowledge base ready')


Processing docs/SIRA_CV_EN.pdf
	7 pages found
	15 splits generated
Saving splits to Vectore Store...
	Saved.

Knowledge base ready


## Query the LLM using Knowledge base as Context

To do so we need:
- an LLM (already created at the beginning)
- the prompt which include both the user input and the context

The prompt can be created through te ChatPromptTemplate module

In [17]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

# https://smith.langchain.com/hub/rlm/rag-prompt
# prompt = hub.pull("rlm/rag-prompt")

prompt = ChatPromptTemplate.from_template("""
Answer the user's question trying to be as synthetic as possible. Use at max 10 words:
Context: {context}
Question: {input}
""")

chain = create_stuff_documents_chain(
    llm = llm,
    prompt = prompt
)

response = chain.invoke({"context": [], "input": 'Which project is Antonino Sirchia working at the moment?'})
print(response)

I'm unable to provide real-time information on current projects.


In [18]:
from langchain.chains import create_retrieval_chain

retriever = vector_store.as_retriever()

retrieval_chain = create_retrieval_chain(retriever, chain)

response = retrieval_chain.invoke({'input': 'Which project is Antonino Sirchia working at the moment?'})
print(response['answer'])

Antonino Sirchia is currently working on ELCA - EasyGov.
